In [1]:
import ipyvolume as ipv
import numpy as np

In [2]:
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))

In [4]:
ipv.__version__

'0.6.0-alpha.8'

In [6]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0

ipv.figure()
ipv.volshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)
ipv.view(-30, 40)
ipv.show()

2021-05-10 09:22:09,723 [219] WARNING  py.warnings:99: [JupyterRequire] /opt/conda/lib/python3.7/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)



In [7]:
fig = ipv.figure()
scatter = ipv.scatter(x, y, z)
ipv.show()

In [2]:
# !pip install --user vaex

     |████████████████████████████████| 42 kB 541 kB/s  eta 0:00:01
     |████████████████████████████████| 2.4 MB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 462 kB/s  eta 0:00:01
     |████████████████████████████████| 106 kB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.7 MB 20.1 MB/s eta 0:00:01     |████████████████████████████▌   | 10.4 MB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 21.8 MB 41 kB/s  eta 0:00:01    |████████▊                       | 5.9 MB 8.5 MB/s eta 0:00:02
     |████████████████████████████████| 3.4 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 260 kB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 23.8 MB/s eta 0:00:01


  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=4f9f5a7b2b775175fd2806fd77f6e35c73821f76a6af7bc30e84c1358451fc3c
  Stored in directory: /home/wfct0p/.cache/pip/wheels/68/17/69/ac196dd181e620bba5fae5488e4fd6366a7316dce13cf88776
  Created wheel for aplus: filename=aplus-0.11.0-py3-none-any.whl size=4412 sha256=49240f2e62c051b444304ba8a2f2898ceb4afc7a4ae103603289e0c560c139cf
  Stored in directory: /home/wfct0p/.cache/pip/wheels/31/ec/1c/cdb682cb8c1d3d1dca4214e96b4845d6d775f78c1067f1a31f
Successfully built frozendict aplus
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.17.0
    Uninstalling pyarrow-0.17.0:
      Successfully uninstalled pyarrow-0.17.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an EnvironmentError: [Errno 16] Device or resource busy: '.nfs0000000980acf22f00000ffc'



In [5]:
import vaex
import numpy as np
import vaex.jupyter.model as vjm



ModuleNotFoundError: No module named 'vaex'

In [4]:
import ipyvolume as ipv
import bqplot

class IpyvolumeBarChart:
    def __init__(self, x_axis, y_axis):
        self.x_axis = x_axis
        self.y_axis = y_axis
        self.fig = ipv.figure()
        self.color_scale = bqplot.ColorScale(scheme='Reds', min=0, max=1)
        ipv.style.set_style_dark()
        ipv.style.box_off()
        ipv.style.use({'axes': {'y': {'visible': False}}})
        self.scatter = None

    def _scale_change(self, change):
        self.last1 = self.x_axis.max
        self.last2 = self.fig.scales['x'].max
        self.x_axis.min = self.fig.scales['x'].min
        self.x_axis.max = self.fig.scales['x'].max
        self.y_axis.min = self.fig.scales['z'].min
        self.y_axis.max = self.fig.scales['z'].max

    def __call__(self, da):
        ar = da.data
        assert ar.ndim == 2
        dim_x = da.dims[0]
        dim_y = da.dims[1]

        Nx, Ny = ar.shape
        x0, x1 = da.coords[dim_x].attrs['min'], da.coords[dim_x].attrs['max']
        y0, y1 = da.coords[dim_y].attrs['min'], da.coords[dim_y].attrs['max']
        x = np.linspace(x0, x1, Nx)
        y = np.linspace(y0, y1, Ny)

        X, Y = np.meshgrid(x, y, indexing='ij')
        xf = X.flatten()
        yf = Y.flatten()
        ar = np.log1p(ar)
        zf = ar.flatten().astype('f8')

        self.dx = dx = x[1] - x[0]
        self.dy = dy = y[1] - y[0]
        if self.scatter is None:
            with self.fig:
                self.scatter = ipv.scatter(xf, 0, yf, aux=zf,
                                           color=zf,
                                           marker="box",
                                           size=1,
                                           color_scale=self.color_scale,
                                           size_x_scale=self.fig.scales['x'],
                                           size_y_scale=self.fig.scales['y'],
                                           size_z_scale=self.fig.scales['z'])
            self.scatter.shader_snippets = {'size': 'size_vector.y = SCALE_SIZE_Y(aux_current) - SCALE_SIZE_Y(0.0) ; '}
            # since we see the boxes with negative sizes inside out, we made the material double sided
            self.scatter.material.side = "DoubleSide"
            ipv.xlim(x0, x1)
            ipv.zlim(y0, y1)
            # only start observing now that the limits have been set to avoid an initial re-gridding
            for scale in [self.fig.scales['x'], self.fig.scales['z']]:
                scale.observe(self._scale_change, ['min', 'max'])
        else:
            with self.scatter.hold_sync():
                self.scatter.x = xf
                self.scatter.z = yf
                self.scatter.aux = zf
                self.scatter.color = zf
        # we patch holes by making the boxes larger
        patch = 1.05
        self.scatter.geo_matrix = [dx*patch, 0, 0, 0,   0, 1, 0, 0,   0, 0, dy*patch, 0,  0.0, 0.5, 0, 1]
        self.color_scale.max = zf.max().item()
        with self.fig:
            # make the x and z lim half a 'box' larger
            ipv.xlim(x0, x1)
            ipv.zlim(y0, y1)
            ipv.ylim(0, zf.max() * 1.2)
            ipv.xlabel(dim_x)
            ipv.zlabel(dim_y)
            ipv.ylabel('counts')
# barchart3d = IpyvolumeBarChart(x_axis, y_axis)


ModuleNotFoundError: No module named 'ipyvolume'

In [ ]:
# x_limits, y_limits = limits = df.limits([df.pickup_longitude, df.pickup_latitude], '95%')

# pre calculated values:
x_limits, y_limits = [-74.37857997, -73.53860359], [40.49456025, 40.91851631]


In [ ]:
# Data is hosted on S3
# df = vaex.open('s3://vaex/taxi/yellow_taxi_2009_2015_f32.hdf5?anon=true')[:800_000_000]

# Or use the Vaex DataFrame server to do the computations!
df = vaex.open('ws://dataframe.vaex.io:80/yellow_taxi_2009_2015_f32')

x_axis = vjm.Axis(df=df, expression=df.pickup_longitude, min=x_limits[0], max=x_limits[1])
y_axis = vjm.Axis(df=df, expression=df.pickup_latitude, min=y_limits[0], max=y_limits[1])

barchart3dtaxi = IpyvolumeBarChart(x_axis, y_axis)

# we use `barchart3d` as a callable function
da_view = df.widget.data_array(axes=[x_axis, y_axis], display_function=barchart3dtaxi, shape=400)

# we display the progress bar and possible output (stack traces)
display(da_view)

# and the figure widget, display(barchart3d.fig) would also have worked
with barchart3dtaxi.fig:
    ipv.show()


In [ ]:
await vaex.jupyter.gather()